In [30]:
import pandas as pd

import sys
import json
import platform
import os,requests
from pathlib import Path
import glob

from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
from src.DataValidationContext import GEDataValidationContext
from src.BatchRequest import getBatchRequest 
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest

from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col,lit,call_udf,countDistinct,sproc,udf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
import pandas as pd
from configs.config import snowflake_conn_prop_local as snowflake_conn_prop
import sys
import json
import platform
import os,requests
from pathlib import Path
import glob
from src.DataValidationContext import *


from snowflake.snowpark import version
print(version.VERSION)
session = Session.builder.configs(snowflake_conn_prop).create()

table = 'CONTACTS'
table_name = 'CONTACTS'
from snowflake.snowpark.functions import col
session = Session.builder.configs(snowflake_conn_prop).create()
from snowflake.snowpark.session import Session


# Retrieve the expectations from the table
df_sql = session.table("CITIBIKE_2.VALIDATION.EXPECTATIONS").filter(col("TABLE_NAME") == table.upper())
data = df_sql.collect()
expectations_df = pd.DataFrame(data)
print(data)

if not expectations_df.empty:
    for _, row in expectations_df.iterrows():
        column_name = row['COLUMN_NAME']
        print(f"column_name: '{column_name}'")                
        expectation_method_name = row['EXPECTATION']
        print(f"expectation_method_name: '{expectation_method_name}'")                
        parameters = json.loads(row['PARAMETERS'])
        print(f"parameters: '{parameters}'")                
        expected_values = parameters.get("expectedValues", [])
        print(f"expected_values: '{expected_values}'")                


(1, 11, 1)
[Row(DB_NAME='CITIBIKE_2', SCHEMA_NAME='NEW_SCHEMA', TABLE_NAME='CONTACTS', COLUMN_NAME='NONE', EXPECTATION='expect_table_column_count_to_equal', PARAMETERS='{"expectedValues": {"value": [5]}}'), Row(DB_NAME='CITIBIKE_2', SCHEMA_NAME='NEW_SCHEMA', TABLE_NAME='CONTACTS', COLUMN_NAME='NONE', EXPECTATION='expect_table_columns_to_match_ordered_list', PARAMETERS='{"expectedValues": {"list": ["FIRSTNAME", "LASTNAME", "AGE"]}}')]
column_name: 'NONE'
expectation_method_name: 'expect_table_column_count_to_equal'
parameters: '{'expectedValues': {'value': [5]}}'
expected_values: '{'value': [5]}'
column_name: 'NONE'
expectation_method_name: 'expect_table_columns_to_match_ordered_list'
parameters: '{'expectedValues': {'list': ['FIRSTNAME', 'LASTNAME', 'AGE']}}'
expected_values: '{'list': ['FIRSTNAME', 'LASTNAME', 'AGE']}'
